In [13]:
import pymongo
with open('/Users/divyashah/Downloads/Twitter project/mongouri.txt', 'r') as file:
    mongo_uri = file.read()
type(mongo_uri)

str

In [14]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
uri = mongo_uri

# Create a new client and connect to the server
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)


Pinged your deployment. You successfully connected to MongoDB!


In [16]:
from pymongo import MongoClient

class SearchInMongoDB(object):
    def __init__(self, search_string, collection_name):
        self.client = MongoClient(uri)
        self.db = self.client["twitter_database"]
        self.collection = self.db[collection_name]
        
        self.fuzzy_search_result = self.fuzzy_search_by_name(search_string)
        self.exact_search_result = self.exact_search_by_name(search_string)
        
    def fuzzy_search_by_name(self, search_string):
        # Escape special characters in the search string
        search_string = search_string.replace("'", "''")
        
        # Execute the MongoDB query for fuzzy search
        fuzzy_search_query = {"$or": [
            {"name": {"$regex": search_string, "$options": "i"}},
            {"screen_name": {"$regex": search_string, "$options": "i"}}
        ]}
        
        result = list(self.collection.find(fuzzy_search_query))
        return result
        
    def exact_search_by_name(self, search_string):
        # Escape special characters in the search string
        search_string = search_string.replace("'", "''")
        
        # Execute the MongoDB query for exact search
        exact_search_query = {"$or": [
            {"name": search_string},
            {"screen_name": search_string}
        ]}
        
        result = list(self.collection.find(exact_search_query))
        return result
def get_search_string():
    print("=== Welcome to MongoDB Search ===")
    print("Enter the search string to find users:")
    search_string = input("Search: ")
    print("================================")
    return search_string

def display_search_results(results, search_type):
    if results:
        print(f"=== Search Results for {search_type} ===")
        for idx, doc in enumerate(results, 1):
            print(f"Result {idx}:")
            print(doc)
            print("--------------------------------")
        print("================================")
    else:
        print(f"No results found for {search_type}.")

# Replace 'your_collection_name' with the actual collection name
collection_name = "users"

# Get the search string from the user
search_string = get_search_string()

# Create an instance of SearchInMongoDB
mongo_search = SearchInMongoDB(search_string, collection_name)

# Display fuzzy search results
display_search_results(mongo_search.fuzzy_search_result, "Fuzzy Search")

# Display exact search results
display_search_results(mongo_search.exact_search_result, "Exact Search")
    
# # Replace 'your_collection_name' with the actual collection name
# collection_name = "users"

# # Provide the search string
# search_string = "Uau"

# # Create an instance of SearchInMongoDB
# mongo_search = SearchInMongoDB(search_string, collection_name)

# print("HERE IS THE RESULT FOR FUZZY SEARCH")
# for doc in mongo_search.fuzzy_search_result:
#     print(doc)

# print("NEXT IS THE RESULT FOR EXACT SEARCH")
# for doc in mongo_search.exact_search_result:
#     print(doc)


=== Welcome to MongoDB Search ===
Enter the search string to find users:


Search:  bi


=== Search Results for Fuzzy Search ===
Result 1:
{'_id': ObjectId('6619cd53bf34cd0b46a5ae5a'), 'created_at': 'Wed Nov 30 19:37:48 +0000 2016', 'description': 'se for da minha família já pode voltar daq mesmo', 'favourites_count': 5446, 'followers_count': 89, 'friends_count': 173, 'id_str': '804046791348015107', 'location': 'Acre, Brasil', 'name': 'Bi Sex Uau', 'screen_name': 'B_King69', 'statuses_count': 4728, 'verified': False}
--------------------------------
Result 2:
{'_id': ObjectId('6619cd54bf34cd0b46a5ae60'), 'created_at': 'Fri Jun 26 09:37:39 +0000 2009', 'description': 'Il successo non è mai definitivo, il fallimento non è mai fatale; è il coraggio di continuare che conta\n(Sir Winston Churchill)', 'favourites_count': 22923, 'followers_count': 1165, 'friends_count': 1840, 'id_str': '50993809', 'location': None, 'name': '🇮🇹Henry Whites♥️', 'screen_name': 'Enrico_Bianchi', 'statuses_count': 42671, 'verified': False}
--------------------------------
Result 3:
{'_id': ObjectId('6